In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/22 12:18:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/22 12:18:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
training = spark.read.csv('test1.csv',header=True,inferSchema=True)

In [3]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [6]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['age','Experience'],outputCol='Independant Features')

In [7]:
output = featureassembler.transform(training)

In [8]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independant Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [12]:
Finalized_Data= output.select('Independant Features','Salary')

In [13]:
Finalized_Data.show()

+--------------------+------+
|Independant Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [14]:
from pyspark.ml.regression import LinearRegression
# train_test_split
train_data,test_data = Finalized_Data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independant Features',labelCol='Salary')
regressor = regressor.fit(train_data)

23/11/22 12:37:32 WARN Instrumentation: [8ff546fc] regParam is zero, which might cause numerical instability and overfitting.
23/11/22 12:37:32 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/11/22 12:37:33 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [15]:
# Coeffecents 

regressor.coefficients

DenseVector([-383.9733, 1711.1853])

In [16]:
# Intercept 

regressor.intercept

23998.330550919385

In [17]:
## Prediction 

pred_results = regressor.evaluate(test_data)

In [18]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independant Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000| 17646.07679465793|
|          [23.0,2.0]| 18000|18589.315525876566|
+--------------------+------+------------------+

